<a href="https://colab.research.google.com/github/luca-g97/Master-Thesis/blob/main/Interactive_AI_Playground.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Initialization: Imports and Dependencies

In [ ]:
# @title Click `Show code` in the code cell. { display-mode: "form" }
!export TF_ENABLE_ONEDNN_OPTS=0

# Suppress FutureWarning globally
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

# Check for local host time. If locally hosted, make sure all dependencies are generated
runningLocal = False if 'google.colab' in str(get_ipython()) else True
print("Running locally" if runningLocal else "Running in Colab")

# if(runningLocal):
import sys
import site

# Add the user site-packages directory to the Python path (for local usage especially)
site.addsitedir('/tf/.local/lib/python3.11/site-packages')

Running locally


**If the notebook is run via Colab, download all necessary files from Github if they aren´t downloaded yet. If you run it locally, please provide them yourself to be able to change them at runtime. We recommend using the following docker image for this: https://hub.docker.com/r/lucag97/colab-server**

In [ ]:
#@title Click `Show code` in the code cell. { display-mode: "form" }

import os

# Define the files and directories you expect to be present
expected_files = ['Customizable_RENN.py','Images_HSVRGB.py','Images_MNIST.py','LLM_Small1x1.py', 'LLM_GPT2.py', 'LLM_LSTM.py', 'Widgets.py', 'Datasets']

# Check for each expected file and directory
def checkIfAllFilesArePresent(expected_files):
    for item in expected_files:
        #path = os.path.join("/content/", item)
        path = os.path.join("./", item)
        if not os.path.exists(path):
            print(f'Missing: {path}')
            return False
    return True

#If not locally running and not all files are present
if not ((checkIfAllFilesArePresent(expected_files)) and runningLocal):
    #!rm -rf /content/*
    !rm -rf ./*
    #Requesting the relevant files from Github
    #!git clone "https://github.com/luca-g97/Master-Thesis.git" /content/Interactive-AI-Playground
    !git clone "https://github.com/luca-g97/Master-Thesis.git" ./Interactive-AI-Playground

    #Move the files into the content folder and delete the original folder afterwards
    #!mv /content/Interactive-AI-Playground/Google_Colab-Interactive_AI_Playground/* /content/
    !mv ./Interactive-AI-Playground/Google_Colab-Interactive_AI_Playground/* ./
    #!rm -rf /content/Interactive-AI-Playground/
    !rm -rf ./Interactive-AI-Playground/

Missing: ./Customizable_RENN.py
Cloning into './Interactive-AI-Playground'...
remote: Enumerating objects: 1237, done.
remote: Counting objects: 100% (144/144), done.
remote: Compressing objects: 100% (95/95), done.
remote: Total 1237 (delta 103), reused 89 (delta 49), pack-reused 1093 (from 2)
Receiving objects: 100% (1237/1237), 10.00 MiB | 30.56 MiB/s, done.
Resolving deltas: 100% (795/795), done.


**If something doesn´t work please uncomment the block below and install the specific versions - those were the ones we last tested with.**

In [ ]:
# @title Click `Show code` in the code cell. { display-mode: "form" }

!python3 -m pip install -q --upgrade pip

# # Install specific versions of the libraries
# !pip install -q torch==2.4.1
# !pip install -q keras==3.4.1
# !pip install -q tqdm==4.66.5
# !pip install -q plotly==5.24.1
# !pip install -q ipython==7.34.0

In [ ]:
# @title Click `Show code` in the code cell. { display-mode: "form" }

import timeit

#Libraries related to Torch
import torch
from torch.utils.data import DataLoader
from torch import nn
import torch.nn.functional as F

#Libraries needed for MNIST dataset
from tqdm import tqdm
from keras.datasets import mnist
from keras.utils import to_categorical

#Libraries needed for HSV-RGB
import colorsys

#Libraries needed for LLM dataset
!pip install -q lorem==0.1.1
!pip install -q tiktoken==0.8.0
#!pip install -q numpy==1.26.4 scipy spacy
#!python -m spacy download en_core_web_sm -q
#import spacy
import lorem
import tiktoken
import random
from transformers import GPT2Tokenizer
#!pip install -q datasets
#from datasets import load_dataset
!pip install -q stanza
import stanza
# Suppress logging from stanza
nlp = stanza.Pipeline('en', verbose=False)
stanza.download('en', verbose=False)

if runningLocal:
    import sys
    from subprocess import run
    sys.path.append('/tf/.local/lib/python3.11/site-packages')
    run([sys.executable, "-m", "pip", "install", "--upgrade", "pip"], check=True)
    run([sys.executable, "-m", "pip", "install", "-q", "nltk"], check=True)

import nltk
nltk.download('punkt_tab')
from nltk import word_tokenize, sent_tokenize

#Library needed for correct displaying widgets in Google Colab
!pip install -q ipywidgets==7.7.1

#Libraries needed for Visualization
!pip install -q plotly
from IPython.display import display, clear_output
import plotly.graph_objects as go
import plotly.io as pio
pio.renderers.default = 'colab'

#Libraries needed for Zipping
import io
import pandas as pd
!pip install -q pyarrow
import pyarrow as pa
!pip install -q pyarrow-parquet
import pyarrow.parquet as pq
!pip install -q zstandard
import zstandard as zstd

#Set the correct device. Prefer a graphics card (cuda) if possible
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print("Using device type: ", device)

2025-01-05 16:43:33.205873: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-01-05 16:43:33.205952: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-01-05 16:43:33.205975: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-05 16:43:33.213352: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/usr/local/lib/python3.11/dist-packages/torch/_uti

Defaulting to user installation because normal site-packages is not writeable


[nltk_data] Downloading package punkt_tab to /tf/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


ERROR: Could not find a version that satisfies the requirement pyarrow-parquet (from versions: none)
ERROR: No matching distribution found for pyarrow-parquet
Using device type:  cuda


#Initialization: Datasets

**MNIST: Number Classification**

In [ ]:
import Images_MNIST as MNIST
MNIST.initializePackages(mnist, to_categorical, nn, DataLoader, device)
trainSetMNIST, testSetMNIST = MNIST.createTrainAndTestSet()

/usr/local/lib/python3.11/dist-packages/_distutils_hack/__init__.py:33: UserWarning:

Setuptools is replacing distutils.



Created 60000 Trainsamples & 10000 Testsamples


**HSV -> RGB Conversion**

In [ ]:
import Images_HSVRGB as HSVRGB
HSVRGB.initializePackages(colorsys, go, pio, DataLoader, device)
#Set visualize to True if you want to see the 3D-cube containing both the train and test samples
trainSetHSVRGB, testSetHSVRGB = HSVRGB.createTrainAndTestSet(50000, 10000, visualize=False)

Created 50653 Trainsamples & 10000 Testsamples


**Small 1x1: LLM-Sourcecheck**

In [ ]:
import LLM_Small1x1 as Small1x1
Small1x1.initializePackages(random, lorem, device, tiktoken, DataLoader)
small1x1 = Small1x1.createTrainAndTestSet(100)

Created 100 samples for Small1x1


**WikiText2: GPT2-Sourcecheck**

In [ ]:
import LLM_GPT2 as GPT2
GPT2.initializePackages(random, lorem, device, tiktoken, DataLoader, nlp, GPT2Tokenizer, nltk)
# Choose one of the following as needed:
# gpt2train, gpt2test = GPT2.createTrainSet()
# gpt2train, gpt2test = GPT2.createWikiTrainSet("sports")
gpt2train, gpt2test = GPT2.createWikiText2TrainSet()
# gpt2train, gpt2test = GPT2.createEnglishWikiTrainSet("./english_wikipedia/data/train-00021-of-00022-8014350d27e6cde7.parquet")

Created a train set with 77611 sentences


**WikiText2: LSTM-Sourcecheck**

In [ ]:
import LLM_LSTM as LSTM
LSTM.initializePackages(device, DataLoader, nltk)
lstmTrain, lstmTest = LSTM.createTrainAndTestSet()

Created a train set with 77524 sentences


#Initialization

In [ ]:
datasets = {
    "MNIST": (trainSetMNIST, testSetMNIST),
    "HSV-RGB": (trainSetHSVRGB, testSetHSVRGB),
    "Small 1x1": (small1x1[:int(len(small1x1) * 0.8)], small1x1[int(len(small1x1) * 0.8):int(len(small1x1) * 0.9)]),
    "WikiText2 (GPT2)": (gpt2train, gpt2test),
    "WikiText2 (LSTM)": (lstmTrain, lstmTest)
}

**After the first run of the notebook, the samples in the Training tab, get set to 10 somehow. Just rerun the cell below and everything should work again.**

In [ ]:
import Widgets
Widgets.initialize(trainSetMNIST, testSetMNIST, datasets)

**To use your settings, please click on THIS CELL and choose "Runtime -> Run cell and below" from the options menu of Colab**

# Selection Initialization

In [ ]:
# @title Click `Show code` in the code cell. { display-mode: "form" }

LLM_Layers, TransformerBlockLayer = Small1x1.setGPTSettings(Widgets.layerAmountChoice.value, Widgets.learningRateChoice.value, Widgets.epochsChoice.value)
LLM_Layers, TransformerBlockLayer = GPT2.setGPTSettings(Widgets.layerAmountChoice.value, Widgets.learningRateChoice.value, Widgets.epochsChoice.value)

datasetChoice = Widgets.datasetChoice.value

if(datasetChoice == "MNIST"):
    chosenDataSet = MNIST
elif(datasetChoice == "HSV-RGB"):
    chosenDataSet = HSVRGB
elif(datasetChoice == "Small 1x1"):
    chosenDataSet = Small1x1
elif datasetChoice == "WikiText2 (GPT2)":
    chosenDataSet = GPT2
elif datasetChoice == "WikiText2 (LSTM)":
    chosenDataSet = LSTM

seed = ""
if (Widgets.seedChoice.value != "Random"):
    seed = Widgets.seedChoice.value
useBitLinear = Widgets.useBitLinearChoice.value

if(datasetChoice == "Small 1x1" or datasetChoice == "WikiText2 (GPT2)"):
    hidden_sizes = []
    hidden_sizes.append(LLM_Layers[0])
    for _ in range(Widgets.layerAmountChoice.value):
        hidden_sizes.append(TransformerBlockLayer)
    hidden_sizes.append(LLM_Layers[1])
    hidden_sizes = [item for sublist in hidden_sizes for item in sublist]
elif(datasetChoice == "WikiText2 (LSTM)"):
    hidden_sizes = LSTM.get_hidden_sizes(Widgets.layerAmountChoice.value, Widgets.trainSamplesChoice.value)
else:
    hidden_sizes = [(normalLayer.value, normalLayerSize.value, activationLayer.value) for normalLayer, normalLayerSize, activationLayer in zip(Widgets.normalLayerChoice, Widgets.normalLayerSizeChoice, Widgets.activationLayerChoice)]
    hidden_sizes.append((Widgets.outputLayerChoice.value, 10, Widgets.outputActivationLayerChoice.value))

visualizeCustom = [((normalLayer.value[0], normalLayer.value[1]), activationLayer.value) for normalLayer, activationLayer in zip(Widgets.neuronChoice, Widgets.activationLayerTypeChoice)]
visualizeCustom.append(((Widgets.outputLayerSizeChoice.value[0], Widgets.outputLayerSizeChoice.value[1]), Widgets.outputLayerActivationChoiceType.value))

epochs = Widgets.epochsChoice.value
learning_rate = Widgets.learningRateChoice.value
train_samples = Widgets.trainSamplesChoice.value
test_samples = Widgets.testSamplesChoice.value
batch_size_training = Widgets.batchSizeTraining.value
batch_size_test = Widgets.batchSizeTest.value
loss_function = Widgets.lossChoice.value
optimizer = Widgets.optimizerChoice.value
eval_samples = Widgets.evalSamplesChoice.value

closestSources = Widgets.closestSourcesChoice.value if Widgets.closestSourcesChoice.value <= train_samples else train_samples
showClosestMostUsedSources = Widgets.showClosestMostUsedSourcesChoice.value
visualizationChoice = Widgets.visualizationChoice.value

In [ ]:
# @title Click `Show code` in the code cell. { display-mode: "form" }

import numpy as np
from collections import defaultdict

dtype = np.dtype([('source', 'U20'), ('value', 'f8'), ('difference', np.float64)])

# Create a compatible fill_value
fill_value = np.array(('None', np.inf, np.inf), dtype=dtype)

#evaluationTest = np.full((train_samples+eval_samples, len(hidden_sizes), np.max([h[1] for h in hidden_sizes]), closestSources),
        #fill_value=fill_value, dtype=dtype)

# Memory usage for each element (in bytes)
element_memory = 3 + 8 + 8  # Source (3 bytes) + value (8 bytes) + difference (8 bytes)

# Initialize total elements to 0
total_non_zero_elements = 0

# Loop through layers and add the non-zero elements per layer
for layer in hidden_sizes:
    neurons_in_layer = layer[1]
    total_non_zero_elements += eval_samples * neurons_in_layer * closestSources

# Total memory (in bytes)
total_storage_bytes = (total_non_zero_elements * 2) * element_memory

# Convert to MB
total_storage_mb = total_storage_bytes / (1024 ** 2)

print(f"Total memory usage for the sparse tensor: {total_storage_mb:.2f} MB")

Total memory usage for the sparse tensor: 33.88 MB


In [ ]:
import Customizable_RENN as RENN
RENN.initializePackages(device, io, pd, pa, pq, zstd, chosenDataSet, seed, useBitLinear)

Initialized Packages for Customizable RENN


# Training

In [ ]:
chosenDataSet.initializeDatasets(train_samples, test_samples, eval_samples, batch_size_training, batch_size_test, seed)
chosenDataSet.trainModel(hidden_sizes, loss_function, optimizer, learning_rate, epochs)

Setting random seed
Created all dataloaders
Model initialized, Starting training
Epoch [1/1000] -> Train-Loss: 7.6130, Train-Perplexity: 2024.3915, Test-Loss: 0.8439, Test-Perplexity: 2.3254
Text generated after epoch 1: The a the in and in of in and of the in and in the to the a in a the to and the a in a and the a the in the to and the to in the to of and the to and a the to a of a the a in of and a in the to and the in of in the of the of the to the in the

Epoch [2/1000] -> Train-Loss: 6.9266, Train-Perplexity: 1019.0376, Test-Loss: 0.8266, Test-Perplexity: 2.2856
Text generated after epoch 2: The the of the a of the a in the in to and of the of a of the a of the to in the of a and the and the in the and the to in the a in the a the to a in a the in of and the in and a of and a to and the to the a to the of the to of in the of the in the to and in

Epoch [3/1000] -> Train-Loss: 6.8206, Train-Perplexity: 916.5409, Test-Loss: 0.8116, Test-Perplexity: 2.2516
Text generated after epoch

# Sourcecheck Customisable RENN

In [ ]:
elapsed_time = timeit.timeit(
    lambda: chosenDataSet.initializeHook(hidden_sizes, train_samples),
    number=1  # Run it once
)
print(f"Time taken: {elapsed_time:.2f} seconds")

Hook-Dictionaries created
Saving all Activations for Training-Source 0 (Actual Training-Source: 0:0)
Saving all Activations for Training-Source 1 (Actual Training-Source: 0:1)
Saving all Activations for Training-Source 2 (Actual Training-Source: 0:2)
Saving all Activations for Training-Source 3 (Actual Training-Source: 0:3)
Saving all Activations for Training-Source 4 (Actual Training-Source: 0:4)
Saving all Activations for Training-Source 5 (Actual Training-Source: 0:5)
Saving all Activations for Training-Source 6 (Actual Training-Source: 0:6)
Saving all Activations for Training-Source 7 (Actual Training-Source: 0:7)
Saving all Activations for Training-Source 8 (Actual Training-Source: 0:8)
Saving all Activations for Training-Source 9 (Actual Training-Source: 0:9)
Saving all Activations for Training-Source 10 (Actual Training-Source: 0:10)
Saving all Activations for Training-Source 11 (Actual Training-Source: 0:11)
Saving all Activations for Training-Source 12 (Actual Training-Source:

# Visualization Customisable RENN

In [ ]:
elapsed_time = timeit.timeit(
    lambda: chosenDataSet.visualize(hidden_sizes, closestSources, showClosestMostUsedSources, visualizationChoice, visualizeCustom, True),
    number=1  # Run it once
)
print(f"Time taken: {elapsed_time:.2f} seconds")

Saving all Activations for Evaluation/Generated-Source 0 (Actual Evaluation/Generated-Source: 0:0)
Saving all Activations for Evaluation/Generated-Source 1 (Actual Evaluation/Generated-Source: 0:1)
Saving all Activations for Evaluation/Generated-Source 2 (Actual Evaluation/Generated-Source: 0:2)
Saving all Activations for Evaluation/Generated-Source 3 (Actual Evaluation/Generated-Source: 0:3)
Saving all Activations for Evaluation/Generated-Source 4 (Actual Evaluation/Generated-Source: 0:4)
Saving all Activations for Evaluation/Generated-Source 5 (Actual Evaluation/Generated-Source: 0:5)
Saving all Activations for Evaluation/Generated-Source 6 (Actual Evaluation/Generated-Source: 0:6)
Starting I/O-bound tasks for Evaluation-Sample 0 (Actual Evaluation-Source: 0:0)
Starting I/O-bound tasks for Evaluation-Sample 1 (Actual Evaluation-Source: 0:1)
Starting I/O-bound tasks for Evaluation-Sample 2 (Actual Evaluation-Source: 0:2)
Starting I/O-bound tasks for Evaluation-Sample 3 (Actual Evaluat

**Only for HSV-RGB conversion: The visualization cell (cell above) must be run before, otherwise there will nothing be displayed!**

In [ ]:
# @title Click `Show code` in the code cell. { display-mode: "form" }

if(datasetChoice == "HSV-RGB"):
    layerChoiceSlider = Widgets.createIntSlider(value=0, min=0, max=len(hidden_sizes)-1, description="Layer")
    neuronChoiceSlider = Widgets.createIntSlider(value=0, min=0, max=hidden_sizes[layerChoiceSlider.value][1]-1, description="Neuron")

    def update_sliders(*args):
        neuronChoiceSlider.max = hidden_sizes[layerChoiceSlider.value][1]-1
        clear_output()
        display(layerChoiceSlider, neuronChoiceSlider)

    layerChoiceSlider.observe(update_sliders, names='value')
    neuronChoiceSlider.observe(update_sliders, names='value')

    display(layerChoiceSlider, neuronChoiceSlider)

**Only for HSV-RGB conversion: Pick the layer and neuron you want to be displayed, then run the cell below to display it**

In [ ]:
# @title Click `Show code` in the code cell. { display-mode: "form" }

if(datasetChoice == "HSV-RGB"):
    def update_visualization(*args):
        clear_output()
        chosenDataSet.visualize3DCube(closestSources, layerChoiceSlider.value, neuronChoiceSlider.value, hidden_sizes[layerChoiceSlider.value][1]-1)

    #Observing causes problems here, which are not fixable somehow (Windows disappearing)
    #layerChoiceSlider.observe(update_visualization, names='value')
    #neuronChoiceSlider.observe(update_visualization, names='value')

    update_visualization()